This notebook explores the data at hand to understand the distribution and shape, as well as perform some pca and UMAP analysis to understand the data better.

In [1]:
import pathlib
from pprint import pprint

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyarrow.parquet as pq
import seaborn as sns
import umap
from sklearn.decomposition import PCA

In [ ]:
figure_dir = pathlib.Path("../figures").resolve()
output_data_dir = pathlib.Path("../data").resolve()
figure_dir.mkdir(parents=True, exist_ok=True)
output_data_dir.mkdir(parents=True, exist_ok=True)

data_subset = False

In [3]:
input_file_path_subset = pathlib.Path(
    "../../4.processing_profiled_features/data/preprocessed_data/live_cell_pyroptosis_wave1_sc_first_time_norm_fs_subset.parquet"
).resolve()
input_file_path = pathlib.Path(
    "../../4.processing_profiled_features/data/feature_selected_data/live_cell_pyroptosis_wave1_sc_first_time_norm_fs.parquet"
).resolve()
agg_file_path = pathlib.Path(
    "../../4.processing_profiled_features/data/aggregated/live_cell_pyroptosis_wave1_first_time_norm_fs_agg.parquet"
).resolve()
output_data_dir = pathlib.Path("../data/first_time").resolve()
figure_dir = pathlib.Path("../figures/first_time").resolve()

In [4]:
random_state = 0
umap_reducer = umap.UMAP(
    n_neighbors=15, min_dist=0.1, n_components=2, random_state=random_state
)
max_pca_components = 100
scree_pca = PCA(n_components=max_pca_components)
pca_reducer = PCA(n_components=2, random_state=random_state)

if data_subset:
    data_df = pd.read_parquet(input_file_path_subset)
else:
    data_df = pd.read_parquet(input_file_path)
# set the UMAP parameters
# separate the data into features and labels
metadata_columns = data_df.columns[data_df.columns.str.contains("Metadata")]
metadata_columns_df = data_df[metadata_columns]
features_df = data_df.drop(metadata_columns, axis=1)
features_df = features_df.fillna(0)
print(
    f"Original data shape: {data_df.shape}, features shape: {features_df.shape}, and metadata shape: {metadata_columns_df.shape}"
)

# fit the UMAP model
umap_embedding = umap_reducer.fit_transform(features_df)
umap_embedding_df = pd.DataFrame(umap_embedding, columns=["UMAP0", "UMAP1"])
umap_embedding_df = pd.concat(
    [metadata_columns_df.reset_index(drop=True), umap_embedding_df], axis=1
)

umap_embeddings_file_path = pathlib.Path(
    output_data_dir / "umap_embeddings.parquet"
).resolve()
umap_embedding_df.to_parquet(umap_embeddings_file_path)
print(f"UMAP embedding shape: {umap_embedding_df.shape}")
umap_embedding_df.head()
# scree plot analysis
scree_pca.fit(features_df)
pca_variance = scree_pca.explained_variance_ratio_

scree_plot_file_path = pathlib.Path(output_data_dir / "scree_plot.parquet").resolve()
scree_plot_df = pd.DataFrame(pca_variance, columns=["Explained Variance"])
scree_plot_df["Principal Component"] = range(1, max_pca_components + 1)
scree_plot_df.to_parquet(scree_plot_file_path)

# perform PCA on the data
pca_embedding = pca_reducer.fit_transform(features_df)
pca_embedding_df = pd.DataFrame(pca_embedding, columns=["PCA0", "PCA1"])
pca_embedding_df = pd.concat(
    [metadata_columns_df.reset_index(drop=True), pca_embedding_df], axis=1
)

pca_embeddings_file_path = pathlib.Path(
    output_data_dir / "pca_embeddings.parquet"
).resolve()
pca_embedding_df.to_parquet(pca_embeddings_file_path)
print(f"PCA embedding shape: {pca_embedding_df.shape}")
pca_embedding_df.head()

Original data shape: (2300, 709), features shape: (2300, 677), and metadata shape: (2300, 32)


/home/lippincm/miniforge3/envs/pyroptosis_timnelapse_env/lib/python3.12/site-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


UMAP embedding shape: (2300, 34)
PCA embedding shape: (2300, 34)


,Metadata_treatment,Metadata_Well,Metadata_number_of_singlecells,Metadata_FOV,Metadata_treatment1,Metadata_Time,Metadata_treatment2,Metadata_treatment1_dose,Metadata_treatment1_unit,Metadata_treatment2_dose,...,Metadata_Image_FileName_GSDM,Metadata_Image_PathName_BF,Metadata_Image_PathName_CL488,Metadata_Image_PathName_CL561,Metadata_Image_PathName_DNA,Metadata_Image_PathName_GSDM,Metadata_Nuclei_Location_Center_X,Metadata_Nuclei_Location_Center_Y,PCA0,PCA1
0,Media,C04,479,0002,Media,01,None,None,None,None,...,T01_W0052_F0002_C1_illumcorrect.tiff,/gpfs/alpine1/scratch/mlippincott@xsede.org/py...,/gpfs/alpine1/scratch/mlippincott@xsede.org/py...,/gpfs/alpine1/scratch/mlippincott@xsede.org/py...,/gpfs/alpine1/scratch/mlippincott@xsede.org/py...,/gpfs/alpine1/scratch/mlippincott@xsede.org/py...,1514.905660,1318.993711,-10.412508,-5.236274
1,Media,C04,490,0002,Media,03,None,None,None,None,...,T03_W0052_F0002_C1_illumcorrect.tiff,/gpfs/alpine1/scratch/mlippincott@xsede.org/py...,/gpfs/alpine1/scratch/mlippincott@xsede.org/py...,/gpfs/alpine1/scratch/mlippincott@xsede.org/py...,/gpfs/alpine1/scratch/mlippincott@xsede.org/py...,/gpfs/alpine1/scratch/mlippincott@xsede.org/py...,817.497792,1742.416667,-12.268307,-5.136458
2,Media,C04,653,0001,Media,08,None,None,None,None,...,T08_W0052_F0001_C1_illumcorrect.tiff,/gpfs/alpine1/scratch/mlippincott@xsede.org/py...,/gpfs/alpine1/scratch/mlippincott@xsede.org/py...,/gpfs/alpine1/scratch/mlippincott@xsede.org/py...,/gpfs/alpine1/scratch/mlippincott@xsede.org/py...,/gpfs/alpine1/scratch/mlippincott@xsede.org/py...,1716.056291,1026.743377,-11.431937,-5.619558
3,Media,C04,838,0001,Media,15,None,None,None,None,...,T15_W0052_F0001_C1_illumcorrect.tiff,/gpfs/alpine1/scratch/mlippincott@xsede.org/py...,/gpfs/alpine1/scratch/mlippincott@xsede.org/py...,/gpfs/alpine1/scratch/mlippincott@xsede.org/py...,/gpfs/alpine1/scratch/mlippincott@xsede.org/py...,/gpfs/alpine1/scratch/mlippincott@xsede.org/py...,1439.479583,956.009557,35.382224,3.003036
4,Media,C04,948,0005,Media,16,None,None,None,None,...,T16_W0052_F0005_C1_illumcorrect.tiff,/gpfs/alpine1/scratch/mlippincott@xsede.org/py...,/gpfs/alpine1/scratch/mlippincott@xsede.org/py...,/gpfs/alpine1/scratch/mlippincott@xsede.org/py...,/gpfs/alpine1/scratch/mlippincott@xsede.org/py...,/gpfs/alpine1/scratch/mlippincott@xsede.org/py...,1885.821145,998.051982,-3.246812,-10.339209


### Process agg

In [5]:
umap_reducer = umap.UMAP(
    n_neighbors=15, min_dist=0.1, n_components=2, random_state=random_state
)
max_pca_components = 100
scree_pca = PCA(n_components=max_pca_components)
pca_reducer = PCA(n_components=2, random_state=random_state)

agg_df = pd.read_parquet(agg_file_path)
# set the UMAP parameters
# separate the data into features and labels
metadata_columns = agg_df.columns[agg_df.columns.str.contains("Metadata")]
metadata_columns_df = agg_df[metadata_columns]
features_df = agg_df.drop(metadata_columns, axis=1)
features_df = features_df.fillna(0)
print(
    f"Original data shape: {agg_df.shape}, features shape: {features_df.shape}, and metadata shape: {metadata_columns_df.shape}"
)

# fit the UMAP model
umap_embedding = umap_reducer.fit_transform(features_df)
umap_embedding_df = pd.DataFrame(umap_embedding, columns=["UMAP0", "UMAP1"])
umap_embedding_df = pd.concat(
    [metadata_columns_df.reset_index(drop=True), umap_embedding_df], axis=1
)
output_data_dir = pathlib.Path("../data/first_time").resolve()
umap_embeddings_file_path = pathlib.Path(
    output_data_dir / "aggregate_umap_embeddings.parquet"
).resolve()
umap_embedding_df.to_parquet(umap_embeddings_file_path)
print(f"UMAP embedding shape: {umap_embedding_df.shape}")
umap_embedding_df.head()
# scree plot analysis
scree_pca.fit(features_df)
pca_variance = scree_pca.explained_variance_ratio_

scree_plot_file_path = pathlib.Path(output_data_dir / "scree_plot.parquet").resolve()
scree_plot_df = pd.DataFrame(pca_variance, columns=["Explained Variance"])
scree_plot_df["Principal Component"] = range(1, max_pca_components + 1)
scree_plot_df.to_parquet(scree_plot_file_path)

# perform PCA on the data
pca_embedding = pca_reducer.fit_transform(features_df)
pca_embedding_df = pd.DataFrame(pca_embedding, columns=["PCA0", "PCA1"])
pca_embedding_df = pd.concat(
    [metadata_columns_df.reset_index(drop=True), pca_embedding_df], axis=1
)

pca_embeddings_file_path = pathlib.Path(
    output_data_dir / "aggregate_pca_embeddings.parquet"
).resolve()
pca_embedding_df.to_parquet(pca_embeddings_file_path)
print(f"PCA embedding shape: {pca_embedding_df.shape}")
pca_embedding_df.head()

Original data shape: (8616, 689), features shape: (8616, 677), and metadata shape: (8616, 12)


/home/lippincm/miniforge3/envs/pyroptosis_timnelapse_env/lib/python3.12/site-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(
/home/lippincm/miniforge3/envs/pyroptosis_timnelapse_env/lib/python3.12/site-packages/sklearn/manifold/_spectral_embedding.py:455: UserWarning: Exited postprocessing with accuracies 
[2.01293710e-15 3.83938167e-06 2.76918360e-06 6.96811362e-06]
not reaching the requested tolerance 6.616115570068359e-06.
  _, diffusion_map = lobpcg(


UMAP embedding shape: (8616, 14)
PCA embedding shape: (8616, 14)


,Metadata_treatment,Metadata_Well,Metadata_number_of_singlecells,Metadata_FOV,Metadata_treatment1,Metadata_Time,Metadata_treatment2,Metadata_treatment1_dose,Metadata_treatment1_unit,Metadata_treatment2_dose,Metadata_treatment2_unit,Metadata_serum,PCA0,PCA1
0,Ab1-42 0.4 uM,N11,652,0002,Ab1-42,01,None,0.4,uM,None,None,NuSerum,-2.524399,-2.17619
1,Ab1-42 0.4 uM,N11,668,0003,Ab1-42,01,None,0.4,uM,None,None,NuSerum,-2.524399,-2.17619
2,Ab1-42 0.4 uM,N11,573,0004,Ab1-42,01,None,0.4,uM,None,None,NuSerum,-2.524399,-2.17619
3,Ab1-42 0.4 uM,N11,628,0001,Ab1-42,01,None,0.4,uM,None,None,NuSerum,-2.524399,-2.17619
4,Ab1-42 0.4 uM,N11,582,0005,Ab1-42,01,None,0.4,uM,None,None,NuSerum,-2.524399,-2.17619
